# Ej CNN

## Import libraries 

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.11.0'

## Data processing

### Test dataset 

In [6]:
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

train_dataset = train_datagen.flow_from_directory(
    './dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary',
)

Found 8000 images belonging to 2 classes.


### Train dataset 

In [7]:
# test is only rescaled to assess performance as in production
test_datagen = ImageDataGenerator(
    rescale = 1. / 255
)

test_dataset = test_datagen.flow_from_directory(
    './dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary',
)

Found 2000 images belonging to 2 classes.


## Build CNN 

### Initialize 

In [8]:
cnn = tf.keras.models.Sequential()

2023-05-30 17:59:32.524126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 17:59:32.529361: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### step 1 Convolution 

In [10]:
cnn.add(tf.keras.layers.Conv2D(
    filters = 32, # feature detectors numbers
    kernel_size = 3, # size of the feature detector
    activation = 'relu', # introduces non-linearity
    input_shape = [64, 64, 3] # only needed in first convolution
))

### step 2 Pooling 

In [12]:
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size = 2, # size of the pooling matrix
    strides = 2 # sliding window by
))

### Additional convolutions 

In [13]:
cnn.add(tf.keras.layers.Conv2D(
    filters = 32, # feature detectors numbers
    kernel_size = 3, # size of the feature detector
    activation = 'relu' # introduces non-linearity
))

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size = 2, # size of the pooling matrix
    strides = 2 # sliding window by
))

### step 3 flattening 

In [14]:
cnn.add(tf.keras.layers.Flatten())

### step 4 fully connected layer 

In [15]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### step 5 output layer 

In [16]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Train CNN 

### compile CNN 

In [17]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### training on train dataset and evaluating on test 

In [18]:
cnn.fit(x = train_dataset, validation_data = test_dataset, epochs = 25)

Epoch 1/25
250/250 [==============================] - 116s 459ms/step - loss: 0.6696 - accuracy: 0.5954 - val_loss: 0.6249 - val_accuracy: 0.6565
Epoch 2/25
250/250 [==============================] - 106s 424ms/step - loss: 0.6154 - accuracy: 0.6655 - val_loss: 0.5589 - val_accuracy: 0.7170
Epoch 3/25
250/250 [==============================] - 101s 406ms/step - loss: 0.5660 - accuracy: 0.7045 - val_loss: 0.5344 - val_accuracy: 0.7435
Epoch 4/25
250/250 [==============================] - 105s 419ms/step - loss: 0.5306 - accuracy: 0.7385 - val_loss: 0.4934 - val_accuracy: 0.7595
Epoch 5/25
250/250 [==============================] - 86s 346ms/step - loss: 0.5007 - accuracy: 0.7509 - val_loss: 0.4918 - val_accuracy: 0.7590
Epoch 6/25
250/250 [==============================] - 101s 406ms/step - loss: 0.4776 - accuracy: 0.7700 - val_loss: 0.4687 - val_accuracy: 0.7825
Epoch 7/25
250/250 [==============================] - 97s 388ms/step - loss: 0.4585 - accuracy: 0.7809 - val_loss: 0.4727 - v

## Making single prediction 

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('./dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# preprocess image to fit the network
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = cnn.predict(test_image)
training_dataset.class_indices

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'